# Método de punjo fijo para sistema de ecuaciones no lineales

In [1]:
import numpy as np

def G1(x):
    x1, x2 = x
    return np.array([(1 - x2) / 2, np.sqrt(1 - x1**2)])

def G2(x):
    x1, x2 = x
    return np.array([(1 - x2) / 2, -np.sqrt(1 - x1**2)])

def fixed_point_iteration(G, x0, tol=1e-6, max_iter=1000):
    x = x0
    for i in range(max_iter):
        x_new = G(x)
        if np.linalg.norm(x_new - x) < tol:
            return x_new, i+1
        x = x_new
    return x, max_iter

# Soluciones iniciales
x0 = np.array([0.5, 0.5])

# Aplicar el método de punto fijo para G1
sol_G1, iter_G1 = fixed_point_iteration(G1, x0)
print(f"Solución usando G1: {sol_G1} en {iter_G1} iteraciones")

# Aplicar el método de punto fijo para G2
sol_G2, iter_G2 = fixed_point_iteration(G2, x0)
print(f"Solución usando G2: {sol_G2} en {iter_G2} iteraciones")


Solución usando G1: [9.93018834e-10 1.00000000e+00] en 7 iteraciones
Solución usando G2: [ 0.79999975 -0.59999917] en 68 iteraciones


In [20]:
import numpy as np
from scipy.linalg import eigvals

def G1(x):
    x1, x2 = x
    return np.array([(1 - x2) / 2, np.sqrt(1 - x1**2)])

def G2(x):
    x1, x2 = x
    return np.array([(1 - x2) / 2, -np.sqrt(1 - x1**2)])

def numerical_jacobian(G, x, eps=1e-8):
    n = len(x)
    jacobian = np.zeros((n, n))
    fx = G(x)
    
    for i in range(n):
        x_eps = np.copy(x)
        x_eps[i] += eps
        fx_eps = G(x_eps)
        jacobian[:, i] = (fx_eps - fx) / eps
    print('Jacobian:')
    print(jacobian)
    return jacobian

# Puntos fijos
x_star1 = np.array([0, 1])
x_star2 = np.array([4/5, -3/5])

# Calcular los jacobianos en los puntos fijos usando la diferenciación numérica
JG1_star1 = numerical_jacobian(G1, x_star1)
JG2_star2 = numerical_jacobian(G2, x_star2)

# Calcular los valores propios
eigvals_JG1 = eigvals(JG1_star1)
eigvals_JG2 = eigvals(JG2_star2)

# Calcular la norma espectral (el valor absoluto máximo de los valores propios)
rho_JG1 = max(abs(eigvals_JG1))
rho_JG2 = max(abs(eigvals_JG2))

# Verificar la convergencia
print(f"Norma espectral de JG1 en x*1: {rho_JG1}")
print(f"Norma espectral de JG2 en x*2: {rho_JG2}")

if rho_JG1 < 1:
    print("El método con G1 converge en una vecindad adecuada del punto fijo x*1.")
else:
    print("El método con G1 no converge en una vecindad adecuada del punto fijo x*1.")

if rho_JG2 < 1:
    print("El método con G2 converge en una vecindad adecuada del punto fijo x*2.")
else:
    print("El método con G2 no converge en una vecindad adecuada del punto fijo x*2.")
    
print(G2([0, 1]))


Jacobian:
[[0. 0.]
 [0. 0.]]
Jacobian:
[[ 0.         -0.50000001]
 [ 1.33333335  0.        ]]
Norma espectral de JG1 en x*1: 0.0
Norma espectral de JG2 en x*2: 0.8164965940953578
El método con G1 converge en una vecindad adecuada del punto fijo x*1.
El método con G2 converge en una vecindad adecuada del punto fijo x*2.
[ 0. -1.]


# Método de Newton para sistemas de ecuaciones no lineales

In [17]:
import numpy as np

def F(x):
    x1, x2 = x
    return np.array([x1 + 1/2 * np.log(x1*x2) - 2, x1 * x2**2 + 5*x2 - 3])

def JF(x):
    x1, x2 = x
    return np.array([[1 + 1 / 2*x1, 1 / 2*x2], [x2**2, 2*x1*x2 + 5]])

def newton_system(F, JF, x0, tol=1e-5, max_iter=1000):
    x = x0
    for i in range(max_iter):
        Fx = -F(x)
        JFx = JF(x)
        h = np.linalg.solve(JFx, Fx)
        x_new = x + h
        if np.linalg.norm(h) < tol:
            return x_new, i+1
        x = x_new
    return x, max_iter

# Solución inicial
x0 = np.array([1, 1.5])

# Aplicar el método de Newton
sol, iter = newton_system(F, JF, x0)
print(f"Solución: {sol} en {iter} iteraciones")

Solución: [2.00000396 0.49999986] en 13 iteraciones


# Metodo de Homotopia

In [21]:
import numpy as np

def F(x):
    x1, x2 = x
    return np.array([x1**2 + x2**2 - 1, 2*x1 + x2 - 1])

def G(x):
    return np.array([x[0], x[1]])

def homotopy_H(x, t):
    return t * F(x) + (1 - t) * G(x)

def homotopy_dH_dt(x, t):
    return F(x) - G(x)

def runge_kutta_4(f, x0, t0, t1, dt):
    t = t0
    x = x0
    while t < t1:
        k1 = dt * f(x, t)
        k2 = dt * f(x + 0.5 * k1, t + 0.5 * dt)
        k3 = dt * f(x + 0.5 * k2, t + 0.5 * dt)
        k4 = dt * f(x + k3, t + dt)
        x = x + (k1 + 2*k2 + 2*k3 + k4) / 6
        t = t + dt
        print(f"t: {t}, x: {x}")
    return x

# Parámetros iniciales
x0 = np.array([1.0, 0.0])  # Solución inicial de G(x) = 0
t0 = 0.0
t1 = 1.0
dt = 0.01

# Resolver la homotopía usando Runge-Kutta de cuarto orden
sol = runge_kutta_4(homotopy_dH_dt, x0, t0, t1, dt)
print(f"Solución final: {sol}")


t: 0.01, x: [0.9899505  0.00989967]
t: 0.02, x: [0.97980398 0.01959737]
t: 0.03, x: [0.9695634 0.0290912]
t: 0.04, x: [0.95923167 0.0383793 ]
t: 0.05, x: [0.94881169 0.04745988]
t: 0.060000000000000005, x: [0.9383063 0.0563312]
t: 0.07, x: [0.92771833 0.06499158]
t: 0.08, x: [0.91705056 0.0734394 ]
t: 0.09, x: [0.90630573 0.08167309]
t: 0.09999999999999999, x: [0.89548655 0.08969113]
t: 0.10999999999999999, x: [0.8845957  0.09749207]
t: 0.11999999999999998, x: [0.8736358 0.1050745]
t: 0.12999999999999998, x: [0.86260944 0.11243706]
t: 0.13999999999999999, x: [0.8515192  0.11957845]
t: 0.15, x: [0.84036757 0.12649742]
t: 0.16, x: [0.82915705 0.13319276]
t: 0.17, x: [0.81789007 0.13966333]
t: 0.18000000000000002, x: [0.80656904 0.14590801]
t: 0.19000000000000003, x: [0.79519633 0.15192574]
t: 0.20000000000000004, x: [0.78377426 0.15771553]
t: 0.21000000000000005, x: [0.77230513 0.1632764 ]
t: 0.22000000000000006, x: [0.76079119 0.16860744]
t: 0.23000000000000007, x: [0.74923467 0.1737077

# Broyden

In [23]:
import numpy as np

def F(x):
    x1, x2 = x
    return np.array([x1**2 + x2**2 - 1, 2*x1 + x2 - 1])

def broyden_method(F, x0, tol=1e-6, max_iter=100):
    n = len(x0)
    x = x0
    J = np.eye(n)  # Inicializamos la aproximación del jacobiano como la matriz identidad
    for k in range(max_iter):
        Fx = F(x)
        if np.linalg.norm(Fx) < tol:
            return x, k+1
        s = -np.linalg.solve(J, Fx)
        x_new = x + s
        y = F(x_new) - Fx
        J = J + np.outer((y - J @ s), s) / np.dot(s, s)
        x = x_new
        print(f"Iteración {k+1}: x = {x}, J = {J}")
    return x, max_iter

# Valores iniciales
x0 = np.array([0, 0.5])

# Aplicar el método de Broyden
sol, iter = broyden_method(F, x0)
print(f"Solución final: {sol} en {iter} iteraciones")


Iteración 1: x = [0.75 1.  ], J = [[1.51923077 0.34615385]
 [1.38461538 1.92307692]]
Iteración 2: x = [0.51968504 0.38582677], J = [[1.83027513 1.17560549]
 [1.15700738 1.31612223]]
Iteración 3: x = [ 1.72559592 -0.99736135], J = [[ 2.89472555 -0.04532649]
 [ 1.67763878  0.71895442]]
Iteración 4: x = [ 0.7044076  -0.63662272], J = [[ 2.98049925 -0.07562638]
 [ 1.87597017  0.64889311]]
Iteración 5: x = [ 0.74350304 -0.39857789], J = [[ 2.78678899 -1.2550921 ]
 [ 1.9353775   1.01061332]]
Iteración 6: x = [ 0.77789739 -0.5519444 ], J = [[ 2.6611626  -0.6949164 ]
 [ 1.94073816  0.98670979]]
Iteración 7: x = [ 0.79962573 -0.59858365], J = [[ 2.6423154  -0.65446145]
 [ 1.94621934  0.97494461]]
Iteración 8: x = [ 0.80009373 -0.60020286], J = [[ 2.7071306  -0.87871392]
 [ 1.94368226  0.98372261]]
Iteración 9: x = [ 0.80000828 -0.60001836], J = [[ 2.63420373 -0.72126083]
 [ 1.94742224  0.97564779]]
Iteración 10: x = [ 0.79999995 -0.59999988], J = [[ 2.63892815 -0.73174246]
 [ 1.94717736  0.9761